In [ ]:
# Mount Google Drive

# Install Plyfile

In [ ]:
!pip install plyfile

# Create a Point Cloud Video

In [ ]:
help()


Welcome to Python 3.10's help utility!

If this is your first time using Python, you should definitely check out
the tutorial on the internet at https://docs.python.org/3.10/tutorial/.

Enter the name of any module, keyword, or topic to get help on writing
Python programs and using Python modules.  To quit this help utility and
return to the interpreter, just type "quit".

To get a list of available modules, keywords, symbols, or topics, type
"modules", "keywords", "symbols", or "topics".  Each module also comes
with a one-line summary of what it does; to list the modules whose name
or summary contain a given string such as "spam", type "modules spam".


You are now leaving help and returning to the Python interpreter.
If you want to ask for help on a particular object directly from the
interpreter, you can type "help(object)".  Executing "help('string')"
has the same effect as typing a particular string at the help> prompt.


In [ ]:
import torch

# Store the number of sampled points in a variable.
M = 4096

def FPS(frame, M):
  # Initialize a variable to hold the points that were
  # subsampled.
  subsampled_points = None

  for i in range(M):
    # Store the shape of the frame Tensor in
    # variables.
    N, D = frame.shape

    # Print out the current iteration.
    print("{:d}.".format(i + 1))

    if subsampled_points is not None:
      # If the subsampled_points variable is not None,
      # then initialize a variable to hold the sum of the L2 distances of each
      # point in the frame from all the subsampled points.
      distances_of_remaining_points_from_subsampled_points = []

      for j in range(N):
        # Append the sum of the L2 distances of frame[j] from all the subsampled points.
        distances_of_remaining_points_from_subsampled_points.append(
            torch.sum(
                torch.sqrt(
                    torch.sum(
                        (frame[j].unsqueeze(0) - subsampled_points) ** 2,
                        dim = 1
                        )
                    )
                )
            )

      # Find the index of the point among the remaining points that is the farthest from the
      # points that have been selected already.
      max_distance_point_index = torch.argmax(torch.Tensor(distances_of_remaining_points_from_subsampled_points))

      # Concatenate the selected point with the subsampled_points variable.
      subsampled_points = torch.cat((subsampled_points, frame[max_distance_point_index].unsqueeze(0)))

      # Split the frame Tensor into 3 parts. The first part is made of the first max_distance_point_index
      # points in frame. The second part consists of the point at max_distance_point_index and
      # the third part consists of the rest of the points in frame.
      frame_split_1, closest_point, frame_split_3 = torch.split(frame, [max_distance_point_index, 1, N - 1 - max_distance_point_index])

      # Concatenate the first part and the last part together, reassign frame
      # with the result of concatenation, to remove the selected point from frame.
      frame = torch.cat((frame_split_1, frame_split_3))
    else:
      # If the subsampled_points variable is None, then choose a random point from the first point cloud frame.
      random_index = torch.randint(0, N, (1, )).item()

      # Select the random point from frame and store it in the subsampled_points variable.
      subsampled_points = frame[random_index].unsqueeze(0)

      # Split the frame Tensor into 3 parts. The first part is made of first random_index
      # points in frame. The second part consists of the point at random_index and
      # the third part consists of the rest of the points in frame.
      frame_split_1, random_point, frame_split_3 = torch.split(frame, [random_index, 1, N - 1 - random_index])

      # Concatenate the first part and the last part together, reassign frame
      # with the result of concatenation, to remove the random point from frame.
      frame = torch.cat((frame_split_1, frame_split_3))

  return subsampled_points

In [ ]:
import plyfile
import numpy
import numpy.lib.recfunctions

# Store how many point cloud frames are in a video in a variable.
num_frames_per_video = 1500

# Store the directory that contains the point clouds in a variable.
point_cloud_video_directory = '/content/drive/MyDrive/Colab Notebooks/COMPSCI 674/Final Project/IKEA/ANU_ikea_dataset_point_cloud_video/processed_frames/Lack_Coffee_Table/0004_black_floor_06_03_2019_08_21_11_46/dev3/point_clouds/'

# Store the point cloud video in a variable.
point_cloud_frames = []

# Loop over the frames of a video.
for i in range(num_frames_per_video):
  # Store the name of the file containing a point cloud frame.
  point_cloud_frame_filename = point_cloud_video_directory + "{:0=6d}.ply".format(i)

  # Read a point cloud frame into a variable from a PLY file.
  point_cloud_frame = plyfile.PlyData.read(point_cloud_frame_filename)

  # Store the vertices of the point cloud frame in a variable.

  # FPS is too time consuming.
  #vertices = FPS(torch.Tensor(numpy.lib.recfunctions.structured_to_unstructured(point_cloud_frame.elements[0].data)), M)
  vertices = numpy.lib.recfunctions.structured_to_unstructured(point_cloud_frame.elements[0].data).reshape(3, -1).transpose((1, 0))
  #vertices = vertices[numpy.random.randint(0, vertices.shape[0], size = M)].reshape(1, M, -1)

  # Append the vertices numpy array to the framw list.
  point_cloud_frames.append(vertices)

  # Print out the current information of the loop.
  print("{:d}. Point Cloud Frame has {} element of shape {}".format(i + 1, point_cloud_frame.elements.__len__(), vertices.shape))

1. Point Cloud Frame has 1 element of shape (153331, 3)
2. Point Cloud Frame has 1 element of shape (154562, 3)
3. Point Cloud Frame has 1 element of shape (155044, 3)
4. Point Cloud Frame has 1 element of shape (154772, 3)
5. Point Cloud Frame has 1 element of shape (154839, 3)
6. Point Cloud Frame has 1 element of shape (154650, 3)
7. Point Cloud Frame has 1 element of shape (154659, 3)
8. Point Cloud Frame has 1 element of shape (154876, 3)
9. Point Cloud Frame has 1 element of shape (154791, 3)
10. Point Cloud Frame has 1 element of shape (154581, 3)
11. Point Cloud Frame has 1 element of shape (154964, 3)
12. Point Cloud Frame has 1 element of shape (154776, 3)
13. Point Cloud Frame has 1 element of shape (151230, 3)
14. Point Cloud Frame has 1 element of shape (154129, 3)
15. Point Cloud Frame has 1 element of shape (154151, 3)
16. Point Cloud Frame has 1 element of shape (154275, 3)
17. Point Cloud Frame has 1 element of shape (154506, 3)
18. Point Cloud Frame has 1 element of s

In [ ]:
# Print out the number of point cloud frames.
print(len(point_cloud_frames))

1500


In [ ]:
import pickle

pickle_file_path = '/content/drive/MyDrive/Colab Notebooks/COMPSCI 674/Final Project\
/IKEA/ANU_ikea_dataset_point_cloud_video/processed_frames/Lack_Coffee_Table/0004_black\
_floor_06_03_2019_08_21_11_46/dev3/point_cloud_video.pickle'

# Save the point cloud video to a file
# by pickling it.
with open(
    pickle_file_path,
    'wb'
    ) as pickle_file:
  pickle.dump(point_cloud_frames, pickle_file)

# Extract Action Annotations

In [42]:
with open('/content/drive/MyDrive/Colab Notebooks/\
COMPSCI 674/Final Project/IKEA/action_annotations/\
gt_action.npy', 'rb') as object_array:
  object_bytes = object_array.read()

In [54]:
gt_actions = numpy.load(
    '/content/drive/MyDrive/Colab Notebooks/\
COMPSCI 674/Final Project/IKEA/action_annotations/\
gt_action.npy',
    allow_pickle = True,
    fix_imports = True)
print(gt_actions.dtype, gt_actions.shape)
data_type = gt_actions.dtype
print(data_type.hasobject)
print(data_type.fields)
print(data_type.descr)
print(data_type.itemsize)
print(data_type.metadata)
print(data_type.name)
print(data_type.num)
print(data_type.kind)
print(numpy.can_cast(data_type, numpy.object_))

object ()
True
None
[('', '|O')]
8
None
object
17
O
True
